<a href="https://colab.research.google.com/github/Gus-1003/ISD-invention/blob/main/Pesquisa/4_Teste_Massivo_Modularizado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando Bibliotecas

In [49]:
import os

# A biblioteca "os" fornece funções para interagir com o sistema operacional, como manipulação de caminhos de arquivo, 
# verificação de existência de arquivos/diretórios, criação de diretórios, etc.

import glob

# A biblioteca "glob" é usada para obter uma lista de caminhos de arquivos que correspondem a um determinado padrão. 
# É útil para buscar arquivos em um diretório com base em padrões de nomenclatura ou extensões.

import cv2

# A biblioteca "OpenCV" (Open Source Computer Vision Library) é uma biblioteca popular para processamento de imagem e visão computacional. 
# Ela fornece várias funções e algoritmos para manipulação, análise e processamento de imagens e vídeos.

import numpy as np

# A biblioteca "NumPy" é usada para operações numéricas eficientes em Python. 
# Ela fornece arrays multidimensionais (ndarrays) que são usados para armazenar e manipular dados numéricos.

import pandas as pd

# A biblioteca "Pandas" fornece estruturas de dados e funções para análise de dados. 
# Ela é amplamente utilizada para manipulação e análise de dados tabulares, como planilhas ou bancos de dados.

from google.colab.patches import cv2_imshow

from google.colab import drive

# A biblioteca "drive" permite montar o Google Drive no ambiente do Google Colab, 
# facilitando o acesso a arquivos e diretórios armazenados no Google Drive.

drive.mount('/content/drive/')

# Monta o Google Drive no ambiente do Google Colab, permitindo acessar os arquivos armazenados no Google Drive.

%cd /content/drive/MyDrive/

# Altera o diretório de trabalho atual para o diretório especificado no Google Drive.

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive


# Funções:

## calculate_contour_properties:

In [46]:
def calculate_contour_properties(cnt):
  # Momentos do objeto
  M = cv2.moments(cnt)

  # Calcula a área do contorno
  area = cv2.contourArea(cnt)

  # Calcula o perímetro do contorno
  perimeter = cv2.arcLength(cnt, True)

  # Proporção de aspecto do objeto
  x,y,w,h = cv2.boundingRect(cnt)
  aspect_ratio = float(w)/h
  width = w
  height = h
  center = (int(x), int(y))

  # Forma de achar o contorno 1
  cx = int(x)
  cy = int(y)

  # Forma de achar o contorno 2
  center_x = int(M["m10"] / M["m00"])
  center_y = int(M["m01"] / M["m00"])

  # Ângulo e raio mínimo envolvente em torno do objeto
  rect = cv2.minAreaRect(cnt)
  circle = cv2.minEnclosingCircle(cnt)
  angle = rect[2]
  radius = circle[1]

  box = cv2.boxPoints(rect)
  box = np.int0(box)

  # Extensão do objeto (razão entre a área do objeto e a área do retângulo envolvente mínimo)
  rect_area = w*h
  
  # Calcula a circularidade do contorno
  if perimeter > 0:
      circularity = 4 * np.pi * area / (perimeter ** 2)
  else:
      circularity = 0
      
  return area, perimeter, circularity, radius, x, y, w, h, box, angle, center, cx, cy, center_x, center_y

## process_contours:

In [47]:
def process_contours(img_contours, contours, imagem_original):

  objetos = 0

  for cnt in contours:
    # Aproxima o contorno por uma sequência de segmentos de linha
    approx = cv2.approxPolyDP(cnt, 0.01 * cv2.arcLength(cnt, True), True)

    # Calcula as propriedades do contorno
    area, perimeter, circularity, radius, x, y, w, h, box, angle, center, cx, cy, center_x, center_y = calculate_contour_properties(cnt)

    # Define o raio máximo desejado
    max_radius = 100

    # Limita o raio ao valor máximo
    radius = min(radius, max_radius)

    # Converte o raio para inteiro
    radius = int(radius)

    # Para um padrão Circular:
    if circularity > 0.30 and area > 2000 and radius < 100 and radius > 20:

      # Desenha um contorno Circular
      cv2.circle(img_contours, center, radius, (255, 0, 0), 2)
      
      # Desenho do ponto central
      cv2.circle(img_contours, (center_x, center_y), 5, (255, 0, 0), -1)
      cv2.circle(img_contours, (cx, cy), 5, (0, 255, 0), -1)

      # Contagem total de Objetos Identificados
      objetos = objetos + 1

    else:
      if min(w, h) == 0:
        aspect_ratio = 0
      else:
        aspect_ratio = max(w, h) / min(w, h)

      if aspect_ratio < 2 and area > 2000 and area < 6543.5: 
        # Desenha um contorno quadratico
        cv2.drawContours(img_contours, [box], 0, (0, 0, 255), 2)

        # Desenho do ponto central
        cv2.circle(img_contours, (center_x, center_y), 5, (0, 0, 255), -1)
        cv2.circle(img_contours, (cx, cy), 5, (0, 255, 0), -1)

        # Contagem total de Objetos Identificados
        objetos = objetos + 1

  return imagem_original, objetos

## find_contours:

In [38]:
def find_contours(image):
  # Encontra os contornos na imagem
  contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  return contours

## dilate_image:

In [37]:
def dilate_image(image):
  # Define o kernel para a dilatação
  kernel = np.ones((5, 5), np.uint8)

  # Dilata a imagem
  image = cv2.dilate(image, kernel, iterations=2)
  return image

## apply_threshold:

In [39]:
def apply_threshold(image):
  # Aplica um filtro de Sobel em x e y
  sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
  sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)

  # Calcula a magnitude do gradiente
  gradient_magnitude = cv2.magnitude(sobelx, sobely)

  # Normaliza a magnitude do gradiente
  gradient_magnitude = cv2.normalize(gradient_magnitude, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

  # Aplica uma limiarização utilizando o método de Otsu
  _, gradient_threshold = cv2.threshold(gradient_magnitude, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
  
  return gradient_threshold

## apply_filters:

In [32]:
def apply_filters(image):
  # Converte a imagem para float32
  image32f = np.float32(image)
  
  # Aplica um filtro de média
  mu = cv2.blur(image32f, (3, 3))

  # Calcula a variância da imagem filtrada
  mu2 = cv2.blur(image32f * image32f, (3, 3))
  sigma = cv2.sqrt(mu2 - mu * mu)
  sigma = sigma * 10
  sigma = sigma.astype("uint8")

  # Aplica um filtro de borramento Gaussiano
  sigma = cv2.GaussianBlur(sigma, (5, 5), 0)

  # Aplica o filtro Laplaciano
  laplacian = cv2.Laplacian(sigma, cv2.CV_32F)

  # Converte a saída para o intervalo de 0 a 255
  sigma = cv2.convertScaleAbs(laplacian)

  return sigma

## print_frame_info:

In [33]:
def print_frame_info(nome):
  # Imprime informações do frame
  print(f"================================================================================")
  print(f"================================================================================")
  print(f"Amostra: " + nome)

## Pilar do Processamento:

In [36]:
def process_frame(frame):
  # Carrega a imagem
  im = cv2.imread(frame)
  
  # Divide o caminho do arquivo e obtém o nome do frame
  nome_completo = frame.split('/')
  nome = nome_completo[int(posicao_do_nome_frame)].split('.')

  # Imprime informações do frame
  print_frame_info(nome[0])

  # Cria uma cópia da imagem para desenhar os contornos
  img_contours = im.copy()

  # Converte a imagem para escala de cinza
  image = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

  # Aplica uma série de filtros à imagem
  sigma = apply_filters(image)

  # Aplica uma limiarização à imagem filtrada
  gradient_threshold = apply_threshold(sigma)

  # Dilata a imagem limiarizada
  gradient_threshold = dilate_image(gradient_threshold)

  # Encontra os contornos na imagem dilatada
  contours = find_contours(gradient_threshold)

  # Processa cada contorno encontrado
  resultados = process_contours(img_contours, contours, img_contours)

  return resultados

## Delete_rats

In [ ]:
def extract_frame_from_video(video, destino, cont=1):

## extract_frame_from_video:

In [21]:
def extract_frame_from_video(video, destino, cont=1):

  # Verificar se o caminho do vídeo existe
  if not os.path.exists(video):
    print(f"O vídeo '{video}' não existe.")
    return

  # Abrir o vídeo
  vidcap = cv2.VideoCapture(video)

  # Verificar se o vídeo foi aberto corretamente
  if not vidcap.isOpened():
    print(f"Falha ao abrir o vídeo '{video}'.")
    return

  interval = 60  # intervalo em segundos (1 minuto)

  # Pular frames com base no intervalo definido
  vidcap.set(cv2.CAP_PROP_POS_FRAMES, int(interval * cont * vidcap.get(cv2.CAP_PROP_FPS)))

  # Extrair o nome do vídeo a partir do caminho
  nome_completo = video.split('/')
  nome = nome_completo[int(posicao_do_nome_video)].split('.')
  nome = nome[0]

  # Ler o próximo frame
  success, image = vidcap.read()

  # Verificar se o frame foi lido com sucesso
  if not success:
    print(f"Falha ao ler a imagem {cont} do vídeo '{video}'.")

    # Liberar os recursos utilizados pelo objeto de leitura do vídeo
    vidcap.release()

    return

  # Salvar o frame lido
  cv2.imwrite("{}{}.jpg".format(destino, nome), image)
  print(f"Frame {cont} extraído do vídeo '{nome}' salva com sucesso em '{destino}'.")

  # Liberar os recursos utilizados pelo objeto de leitura do vídeo
  vidcap.release()
  # Fechar todas as janelas abertas pelo OpenCV
  cv2.destroyAllWindows()

# Base do Sistema:

## Endereço dos Videos:

In [22]:
# entrada = glob.glob('/content/drive/Shareddrives/Lab Electrofisiologia III - Dados/Experimentos comportamento III/Experimento 15/Vídeos separados - TR e TT/grey_field_videos/*.mp4')

entrada = glob.glob('/content/drive/Shareddrives/Lab Electrofisiologia III - Dados/Experimentos comportamento III/Experimento 17/Vídeos separados (explo)/campo preto/*.mpg')

# Imprime os elementos da lista com quebras de linha entre cada elemento
print('\n'.join(entrada))

/content/drive/Shareddrives/Lab Electrofisiologia III - Dados/Experimentos comportamento III/Experimento 17/Vídeos separados (explo)/campo preto/Dia2-Explo-CCF504-Contexto4.mpg
/content/drive/Shareddrives/Lab Electrofisiologia III - Dados/Experimentos comportamento III/Experimento 17/Vídeos separados (explo)/campo preto/Dia2-Explo-CCF508-Contexto4.mpg
/content/drive/Shareddrives/Lab Electrofisiologia III - Dados/Experimentos comportamento III/Experimento 17/Vídeos separados (explo)/campo preto/Dia2-Explo-CCF512-Contexto4.mpg


In [23]:
posicao_do_nome_video = input("Posição do nome do video: ") #9

Posição do nome do video: 9


## Endereço das imagens:

In [24]:
destino = '/content/drive/MyDrive/Pesquisa_ISD_IC/Base de Dados/'

#destino = '/content/drive/MyDrive/Pesquisa_ISD_IC/Imagens_grey/'

#destino = '/content/drive/MyDrive/Pesquisa_ISD_IC/Imagens_Infravermelho/'

#destino = '/content/drive/MyDrive/Pesquisa_ISD_IC/Teste_Massivo/'

### Listando Imagens na pasta:

In [ ]:
frames = glob.glob(destino + '*.jpg')
# print('\n'.join(frames))

In [26]:
posicao_do_nome_frame = input("posicao_do_nome_frame:") #6

posicao_do_nome_frame:6


## Extraindo imagens dos videos:

In [ ]:
# extract_frames_from_videos(entrada, destino)

# Algoritmo Principal:

In [ ]:
# Loop que processa cada arquivo na lista de caminhos de arquivos.
for frame in frames:
  resultados = process_frame(frame)

  # Imprime o número de objetos encontrados
  print(resultados[1])

  # Exibe a imagem com os contornos encontrados
  cv2_imshow(resultados[0])